# Test modules using Polytropes #
Test modules with M, rho, P from polytrope solution and using $P = \rho \frac{R}{\mu}T$ as the equation of state

In [ ]:
%pylab ipympl
ifig = 0

import polytrope
from opacity import table, opal
import nuclear
import structure
import scipy.interpolate

# Polytrope and Equation of state #

In [ ]:
n = 3
rho_c = 150 # [g/cm^3]
P_c = 2.5e17 # [dyn/cm^2]
X, Z = 0.7, 0.02 # mass fraction
data = polytrope.solve_polytrope(rho_c, P_c, n, phi_range=(0, 100), atol=1e-6, rtol=1e-6)
m, r, rho, P = polytrope.polytrope_to_cgs(data, rho_c, P_c, n)

X, Z = X + 0*r, Z + 0*r

R = 8.314e7 # in [erg/(g K)]
mu = 4/(5*X + 3 - Z) # unitless

T = P*mu/(rho*R) # in [K]
T_c = T.max()
print(f"Core tempurature: {T_c} in [K]")

rho = numpy.abs(rho)
P = numpy.abs(P)
T = numpy.abs(T)

## opacity ##

In [ ]:
opacity = table.opal_opacity("opacity/tables/GN93hz")
#opacity = table.analytic_opacity

k_R = opacity(rho, T, X, Z) # in (cm^2/g)
mfp = 1/(rho*k_R)

## nuclear network ##

In [ ]:
reactions = nuclear.network.read_network_file("nuclear/networks/pp_branch_I") + nuclear.network.read_network_file("nuclear/networks/cno_branch_I")
ifig += 1
nuclear.plot.network_graph(reactions, show=True, ifig=ifig)

particles, get_dYdt_epsilon, X_metalicity = nuclear.network.build_network(reactions, "./nuclear/reactions")
print(f"particles: {particles}")

X_0 = nuclear.network.initalize_mass_fraction(particles)
Y_0 = nuclear.network.mass_frac_to_molar_abu(particles, X_0)

epsilon = numpy.zeros(shape=T.shape)
for i in range(len(T)):
    dYdt, epsilon_fun = get_dYdt_epsilon(rho[i], T[i])
    epsilon[i] = epsilon_fun(0, Y_0)

L = numpy.zeros(shape=T.shape)
L[1:] = numpy.cumsum(epsilon[1:]*numpy.diff(m))

power_density = rho*epsilon


In [ ]:
"""ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(m/m.max(), r/r.max(), label="radius $y = R$")
pyplot.plot(m/m.max(), rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(m/m.max(), P/P.max(), label="pressure $y = P$")
pyplot.plot(m/m.max(), T/T.max(), label="tempurature $y = T$")
pyplot.plot(m/m.max(), L/L.max(), label="luminosity $y = L$")
pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m/m.max(), label="mass $y = M$")
pyplot.plot(r, rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(r, P/P.max(), label="pressure $y = P$")
pyplot.plot(r, T/T.max(), label="tempurature $y = T$")
pyplot.plot(r, L/L.max(), label="luminosity $y = L$")
pyplot.plot(r, k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(r, power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius [cm]")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)"""

ifig += 1
structure.plot.PrLT(m, [P, r, L, T], ifig=ifig)

ifig += 1
structure.plot.PrLT(m, [P, r, L, T], massCoord=False, ifig=ifig)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, rho)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("rho [g/cm^3]")
pyplot.title("density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, T)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("temperature [K]")
pyplot.title("Temperature")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.semilogy(r, mfp)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mean free path [cm]")
pyplot.title("Opacity: Mean free path of light")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, power_density)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("power density [erg/(s cm^3)]")
pyplot.title("Nuclear power density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mass [g]")
pyplot.title("total mass")
pyplot.show(ifig)

## Nuclear Network Array ##

In [ ]:
Y_0 = numpy.array(Y_0)[:, None] + 0*T[None, :]
t_range = (0, 1*60*60*24*365.25)

Y, status = nuclear.network.solve_radial_network(rho, T, Y_0, get_dYdt_epsilon, t_range)
print("status:", status)

ifig += 1
nuclear.plot.isotope_profile(m, Y, particles, ifig=ifig)

ifig += 1
nuclear.plot.isotope_profile(r, Y, particles, False, False, ifig=ifig)

In [ ]:
Y_prime = copy(Y)

In [ ]:
t_range = (0, 5e5*60*60*24*365.25)

Y = copy(Y_prime)

ifig += 1
plot_args = (particles, True, ifig)
Y, status = nuclear.network.solve_radial_network(rho, T, Y_0, get_dYdt_epsilon, t_range, plot_args, atol=1e-9, rtol=1e-9)
print("status:", status)

In [ ]:
ifig += 1
nuclear.plot.isotope_profile(m, Y, particles, ifig=ifig)

ifig += 1
nuclear.plot.isotope_profile(r, Y, particles, False, False, ifig=ifig)

Y = numpy.abs(Y)
Y_interp = nuclear.network.interpolate_molar_abundance(m, Y)

In [ ]:
k_R = opacity(rho, T, *X_metalicity(Y)) # in (cm^2/g)
mfp = 1/(rho*k_R)

epsilon = numpy.zeros(shape=T.shape)
for i in range(len(T)):
    dYdt, epsilon_fun = get_dYdt_epsilon(rho[i], T[i])
    epsilon[i] = epsilon_fun(0, Y[:, i])

L = numpy.zeros(shape=T.shape)
L[1:] = numpy.cumsum(epsilon[1:]*numpy.diff(m))

power_density = rho*epsilon

"""ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(m/m.max(), r/r.max(), label="radius $y = R$")
pyplot.plot(m/m.max(), rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(m/m.max(), P/P.max(), label="pressure $y = P$")
pyplot.plot(m/m.max(), T/T.max(), label="tempurature $y = T$")
pyplot.plot(m/m.max(), L/L.max(), label="luminosity $y = L$")
pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m/m.max(), label="mass $y = M$")
pyplot.plot(r, rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(r, P/P.max(), label="pressure $y = P$")
pyplot.plot(r, T/T.max(), label="tempurature $y = T$")
pyplot.plot(r, L/L.max(), label="luminosity $y = L$")
pyplot.plot(r, k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(r, power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius [cm]")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)"""

ifig += 1
structure.plot.PrLT(m, [P, r, L, T], ifig=ifig)

ifig += 1
structure.plot.PrLT(m, [P, r, L, T], massCoord=False, ifig=ifig)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, power_density)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("power density [erg/(s cm^3)]")
pyplot.title("Nuclear power density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.semilogy(r, mfp)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mean free path [cm]")
pyplot.title("Opacity: Mean free path of light")
pyplot.show(ifig)

## Stellar Structure ##

In [ ]:
sigma_sb = 5.67037442e-5 # in [erg/(s cm^2 K^4)]

dPrLTdM = structure.equations.get_dPrLTdM(get_dYdt_epsilon, X_metalicity, opacity, Y_interp)

r_R, T_R = 50e9, 5500

M, PrLT, status, error = structure.equations.solve_core(dPrLTdM, m[-1], r_R, T_R)
print("error:", error)

In [ ]:
M_tot = m[-1]
R_est = polytrope.R_sol*(M_tot/polytrope.M_sol)**0.8
L_est = polytrope.L_sol*(M_tot/polytrope.M_sol)**3.5
T_est = (L_est/(4*numpy.pi*R_est**2*sigma_sb))**(1/4)

print(f"M: {M_tot:.3g}, R: {R_est:.3g}, L: {L_est:.3g}, T: {T_est:.3g}")
print(f"M: {M_tot/polytrope.M_sol:.4f}, R: {R_est/polytrope.R_sol:.4f}, L: {L_est/polytrope.L_sol:.4f}, T: {T_est/5500:.4f}")

r_range, T_range, n = (1, R_est*1.5), (1, T_est*1.5), 5
min_v = float("inf")
min_param = None, None
values = numpy.zeros(shape=(n, n))
k = 0
for i, r_R in enumerate(numpy.linspace(*r_range, n)):
    for j, T_R in enumerate(numpy.linspace(*T_range, n)):
        k += 1
        if k % max(1, n**2//100) == 0:
            print(k)
        
        k_R = 0.2*(1 + 0.7)

        P_R = (2/3)*polytrope.G*m[-1]/(r_R**2*k_R)
        L_R = 4*numpy.pi*r_R**2*sigma_sb*T_R**4

        events = [structure.equations.zr, structure.equations.zL]
        PrLT_R = [P_R, r_R, L_R, T_R]
        try:
            solve = scipy.integrate.solve_ivp(dPrLTdM, (m[-1], m[0]), PrLT_R, events=events, method="Radau")
            PrLT = solve["y"]
            M = solve["t"]
            #print(", ".join([f"{v:11.3e}" for v in structure.equations.core_error(M, PrLT)]), solve.message)
            values[i, j] = numpy.sqrt(numpy.sum(numpy.array(structure.equations.core_error(M, PrLT))**2))
            if values[i, j] < min_v:
                min_v = values[i, j]
                min_param = (r_R, T_R)
        except:
            raise
            values[i, j] = float("Nan")

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.imshow(values, extent=(*T_range, *r_range), aspect='auto')
pyplot.show(ifig)
print(min_v, min_param)
print(f"R: {min_param[0]:.3g}, T: {min_param[1]:.3g}")

In [ ]:
M1 = None
M1 = solve["t"]
P1, r1, L1, T1 = solve["y"]
 
Y_m = Y_interp(M1)
X_m = nuclear.network.molar_abu_to_mass_frac(particles, Y_m)

mf_X, mf_Z = X_metalicity(Y_m)
mu = 4/(5*mf_X + 3 - mf_Z) # []

rho1 = mu*P1/(R*T1)
k_R1 = opacity(rho1, T1, mf_X, mf_Z)
mpl1 = 1/(rho1*k_R1)

P_t = (2/3)*polytrope.G*M1[-1]/(r1[-1]**2*k_R1[-1])
T_t = (L1[-1]/(4*numpy.pi*r1[-1]**2*sigma_sb))**(1/4)

"""ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(M1/M1.max(), r1/r1.max(), label="radius $y = R$")
pyplot.plot(M1/M1.max(), rho1/rho1.max(), marker="s", label="density $y = \\rho$")
pyplot.plot(M1/M1.max(), P1/P1.max(), label="pressure $y = P$")
pyplot.plot(M1/M1.max(), T1/T1.max(), marker="*", label="tempurature $y = T$")
pyplot.plot(M1/M1.max(), L1/L1.max(), label="luminosity $y = L$")
#pyplot.plot(M1/M1.max(), k_R1/k_R1.max(), label="opacity")
#pyplot.plot(M1/M1.max(), mpl1/mpl1.max(), label="mpl")

pyplot.hlines(T_t/T1.max(), 0, 1, "r", "--", label="T target")
pyplot.hlines(P_t/P1.max(), 0, 1, "g", ":", label="T target")
#pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
#pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1, M1/M1.max(), label="mass $y = R$")
pyplot.plot(r1, rho1/rho1.max(), marker="s", label="density $y = \\rho$")
pyplot.plot(r1, P1/P1.max(), label="pressure $y = P$")
pyplot.plot(r1, T1/T1.max(), marker="*", label="tempurature $y = T$")
pyplot.plot(r1, L1/L1.max(), label="luminosity $y = L$")
#pyplot.plot(r1, k_R1/k_R1.max(), label="opacity")
#pyplot.plot(r1, mpl1/mpl1.max(), label="mpl")
#pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
#pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius in cm")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)"""

ifig += 1
structure.plot.PrLT(M1, solve["y"], ifig=ifig)

ifig += 1
structure.plot.PrLT(M1, solve["y"], massCoord=False, ifig=ifig)

In [ ]:
k_R = opacity(rho1, T1, mf_X, mf_Z) # in (cm^2/g)
mfp = 1/(rho1*k_R)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1, rho1)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("density [g/cm^3]")
pyplot.title("Density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1/polytrope.R_sol, M1/polytrope.M_sol)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mass [g]")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1/polytrope.R_sol, L1/polytrope.L_sol)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("luminosity [W]")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1, T1)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("Tempurature [K]")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1, numpy.gradient(mfp, r1))
pyplot.xlabel("radius [cm]")
pyplot.ylabel("Mean free path gradient")
pyplot.show(ifig)